In [1]:
import numpy as np
import pandas as pd


đọc file  

In [2]:
df=pd.read_csv('./population.csv',index_col=0)

kiểm tra lặp dòng

In [3]:
len([i for i in df.index.duplicated(keep='first') if i==True])

0

check thông tin của df 

In [4]:
df.head()

,Year,Population,Yearly%Change,YearlyChange,Migrants(net),MedianAge,FertilityRate,Density(P/Km2),UrbanPop%,UrbanPopulation,Country%OfWorldPop,GlobalRank,Country,Continent
0,1955,8270991,1.30,103775,-4000.0,19.2,7.45,13,7.1,587818.0,0.30,48,Afghanistan,Asia
1,1960,8996973,1.70,145196,-4000.0,18.8,7.45,14,8.4,755797.0,0.30,48,Afghanistan,Asia
2,1965,9956320,2.05,191869,-4000.0,18.4,7.45,15,9.9,984350.0,0.30,48,Afghanistan,Asia
3,1970,11173642,2.33,243464,-4000.0,17.9,7.45,17,11.6,1295433.0,0.30,48,Afghanistan,Asia
4,1975,12689160,2.58,303104,-4000.0,17.3,7.45,19,13.5,1717422.0,0.31,48,Afghanistan,Asia


In [5]:
(row_count,col_count)=df.shape
print(row_count)
df.dtypes

4230


Year                    int64
Population              int64
Yearly%Change         float64
YearlyChange            int64
Migrants(net)         float64
MedianAge             float64
FertilityRate         float64
Density(P/Km2)          int64
UrbanPop%             float64
UrbanPopulation       float64
Country%OfWorldPop    float64
GlobalRank              int64
Country                object
Continent              object
dtype: object

In [6]:
df.describe()

,Year,Population,Yearly%Change,YearlyChange,Migrants(net),MedianAge,FertilityRate,Density(P/Km2),UrbanPop%,UrbanPopulation,Country%OfWorldPop,GlobalRank
count,4230.000000,4.230000e+03,4230.000000,4.230000e+03,4.230000e+03,4230.000000,4230.000000,4230.000000,4230.000000,4.230000e+03,4230.000000,4230.000000
mean,1994.166667,2.423329e+07,1.659778,3.271283e+05,-4.121513e+00,21.789243,3.324785,350.130496,52.530544,1.166833e+07,0.425392,117.183452
std,21.723653,1.063217e+08,1.650514,1.329164e+06,9.808264e+04,11.751355,2.289232,1790.415755,26.728939,4.665138e+07,1.800249,68.058075
min,1955.000000,6.440000e+02,-12.930000,-6.730240e+05,-1.077397e+06,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000
25%,1975.000000,2.733908e+05,0.610000,1.201750e+03,-8.997000e+03,17.200000,1.690000,21.000000,31.200000,1.148200e+05,0.000000,58.000000
50%,1997.500000,3.603262e+06,1.530000,3.730850e+04,0.000000e+00,20.700000,2.790000,72.000000,52.500000,1.479994e+06,0.070000,117.000000
75%,2016.000000,1.276953e+07,2.590000,2.038582e+05,2.225250e+03,30.000000,5.347500,173.750000,74.300000,6.585661e+06,0.250000,176.000000
max,2020.000000,1.439324e+09,23.300000,2.067648e+07,1.771991e+06,48.400000,8.800000,26337.000000,104.400000,8.750759e+08,22.700000,235.000000


Kiểm tra tính đúng đắn của một nguồn cực kì tin cậy là không cần thiết
#không có cơ sở để kiểm tra

thêm vào giá trị nan

Cột chứa nan

In [7]:
col_nan=[x for x in df.columns if len([i for i in df[x] if i!=i])>1 ]
col_nan

[]

Các nước nào có dòng nan 

In [8]:
df[df['UrbanPopulation']!=df['UrbanPopulation']]['Country'].unique()

array([], dtype=object)

In [9]:
df[df['MedianAge']!=df['MedianAge']]['Country'].unique()

array([], dtype=object)

In [10]:
df[df['Migrants(net)']!=df['Migrants(net)']]['Country'].unique()


array([], dtype=object)

In [11]:
df[df['FertilityRate']!=df['FertilityRate']]['Country'].unique()


array([], dtype=object)

Có vẻ như các nước thiếu FertilityRate,MedianAge,Migrants(net) đều giống nhau?

In [12]:
print(df[df['FertilityRate']!=df['FertilityRate']]['Country'].unique()==df[df['Migrants(net)']!=df['Migrants(net)']]['Country'].unique())
print(df[df['MedianAge']!=df['MedianAge']]['Country'].unique()==df[df['Migrants(net)']!=df['Migrants(net)']]['Country'].unique())

[]
[]


chia làm 2 loại nan 

In [13]:
missing=df[df['FertilityRate']!=df['FertilityRate']]['Country'].unique()
missing_priority=df[df['UrbanPopulation']!=df['UrbanPopulation']]['Country'].unique()

Xem xét trên df thấy cột UrbanPopulation, UrbanPop% chỉ có thiếu vài giá trị

Kiểm tra nước nào không thể dùng hồi quy để thay thế nan 

In [14]:
for x in [i for i in missing_priority] :
    df_=df[df['Country']==x]
    (r,c)=df_[df_['UrbanPopulation']!=df_['UrbanPopulation']].shape
    print(x,r) if r>16 else 0

Một pha xử lí riêng cho Holy See

In [15]:
HolySee_Percent_Urban=[i for i in df.loc[df['Country']=='Holy See']['UrbanPop%'].values if i==i][0] 
for i in df.loc[df['Country']=='Holy See'].index : 
    df.at[i,'UrbanPop%']=HolySee_Percent_Urban
    df.at[i,'UrbanPopulation']=round(HolySee_Percent_Urban*df.loc[i]['Population']/100)

Hồi quy cho các nước còn lại 

In [16]:
def regressionFillUrban(df,Country):
    df_=df[df['Country']==Country]
    r=df_[df_['UrbanPopulation']!=df_['UrbanPopulation']].index
    df_=df_[df_['UrbanPopulation']==df_['UrbanPopulation']]
    def estimate_coef(x, y):
        n = np.size(x)
        m_x = np.mean(x)
        m_y = np.mean(y)
        SS_xy = np.sum(y*x) - n*m_y*m_x
        SS_xx = np.sum(x*x) - n*m_x*m_x
        b_1 = SS_xy / SS_xx
        b_0 = m_y - b_1*m_x
        return (b_0, b_1)
    def regression(b,x):
        return b[0] + b[1]*x
    b=estimate_coef(df_['Population'],df_['UrbanPopulation'])
    for i in r :
        df.at[i,'UrbanPopulation']=regression(b,df.loc[i]['Population'])
        df.at[i,'UrbanPop%']=round(df.loc[i]['UrbanPopulation']*100/df.loc[i]['Population'],1)
for i in missing_priority:
    regressionFillUrban(df,i)


Xem xét sự phân bố giá trị của FertilityRate,MedianAge,Migrants(net) thấy dữ liệu phân bố đều.
Do đó thay thế bằng giá trị trung vị hoặc trung bình đều không hợp lí.
Các nước có dòng thiếu các giá trị này luôn thiếu cả cột giá trị này.
=> không cần xem xét và bỏ qua.

In [17]:
df=df.fillna(0)

Kiểm tra còn cột nào chứa nan

In [18]:
[x for x in df.columns if len([i for i in df[x] if i!=i])>1 ]

[]

Về logic nếu có diện tích, dân số, ta có thể tính được mật độ dân số 
Ta bỏ đi cột Density(P/Km2)

In [19]:
df.pop('Density(P/Km2)')

0       13
1       14
2       15
3       17
4       19
        ..
4225    36
4226    37
4227    37
4228    38
4229    38
Name: Density(P/Km2), Length: 4230, dtype: int64

In [20]:
df.to_csv('./population.csv')